# Infer Noise Pooled Model Parameters from Individuals in Lung Cancer Control Group

In [36]:
import os

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pints
from scipy.optimize import minimize, basinhopping
from tqdm.notebook import tqdm
import xarray as xr

import erlotinib as erlo

## Show control group data

In [2]:
# Get data
data = erlo.DataLibrary().lung_cancer_control_group()

# Create scatter plot
fig = erlo.plots.PDTimeSeriesPlot()
fig.add_data(data, biomarker='Tumour volume')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')

# Show figure
fig.show()

**Figure 1:** Visualisation of the measured tumour growth in 8 mice with patient-derived lung cancer implants.

## Build model

In [7]:
# Define mechanistic model
path = erlo.ModelLibrary().tumour_growth_inhibition_model_koch_reparametrised()
mechanistic_model = erlo.PharmacodynamicModel(path)
mechanistic_model.set_parameter_names(names={
    'myokit.tumour_volume': 'Tumour volume in cm^3',
    'myokit.critical_volume': 'Critical volume in cm^3',
    'myokit.drug_concentration': 'Drug concentration in mg/L',
    'myokit.kappa': 'Potency in L/mg/day',
    'myokit.lambda': 'Exponential growth rate in 1/day'})
mechanistic_model.set_output_names({
    'myokit.tumour_volume': 'Tumour volume'})

# Define error model
error_model = erlo.ConstantAndMultiplicativeGaussianErrorModel()

# Define population model
population_models = [
    erlo.HeterogeneousModel(),  # Initial tumour volume
    erlo.HeterogeneousModel(),  # Critical volume
    erlo.HeterogeneousModel(),  # Growth rate
    erlo.PooledModel()]         # Sigma rel.

# Compose model and not identified parameters
problem = erlo.ProblemModellingController(mechanistic_model, error_model)
problem.fix_parameters({
    'Drug concentration in mg/L': 0,
    'Potency in L/mg/day': 0,
    'Sigma base': 0.001})
problem.set_population_model(population_models)

## Prior predictive checks

### Population model

In [8]:
# Define prior distribution
log_priors = [
    pints.TruncatedGaussianLogPrior(mean=0.25, sd=0.1, a=0, b=np.inf),   # Initial tumour volume
    pints.TruncatedGaussianLogPrior(mean=1, sd=1, a=0, b=np.inf),        # Critical volume
    pints.TruncatedGaussianLogPrior(mean=0.1, sd=0.1, a=0, b=np.inf),    # Growth rate
    pints.TruncatedGaussianLogPrior(mean=0, sd=1, a=0, b=np.inf)]        # Pooled Sigma rel.
log_prior = pints.ComposedLogPrior(*log_priors)

# Compose prior predictive model
predictive_model = problem.get_predictive_model(exclude_pop_model=True)
prior_predictive_model = erlo.PriorPredictiveModel(
    predictive_model, log_prior)

# Sample from prior predictive model
times = np.linspace(0, 30)
n_samples = 1000
samples = prior_predictive_model.sample(times, n_samples)

# Illustrate prior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples, bulk_probs=[0.3, 0.6, 0.9])
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')
fig.show()

**Figure 3:** Approximate prior predictive model for the tumour growth in a population over time. The shaded areas indicate the 30%, 60% and 90% bulk of the prior predictive model (from dark to light). The prior predictive model was approximated by sampling 1000 parameters from the prior distribution, and subsequent sampling of 50 equidistant time points from the predictive model for each parameter set.

## Find maximum a posteriori estimates

In [37]:
# Create log-posteriors
log_priors = [
    pints.TruncatedGaussianLogPrior(mean=0.25, sd=0.1, a=0, b=np.inf),   # Initial tumour volume
    pints.TruncatedGaussianLogPrior(mean=0.25, sd=0.1, a=0, b=np.inf),   # Initial tumour volume
    pints.TruncatedGaussianLogPrior(mean=0.25, sd=0.1, a=0, b=np.inf),   # Initial tumour volume
    pints.TruncatedGaussianLogPrior(mean=0.25, sd=0.1, a=0, b=np.inf),   # Initial tumour volume
    pints.TruncatedGaussianLogPrior(mean=0.25, sd=0.1, a=0, b=np.inf),   # Initial tumour volume
    pints.TruncatedGaussianLogPrior(mean=0.25, sd=0.1, a=0, b=np.inf),   # Initial tumour volume
    pints.TruncatedGaussianLogPrior(mean=0.25, sd=0.1, a=0, b=np.inf),   # Initial tumour volume
    pints.TruncatedGaussianLogPrior(mean=0.25, sd=0.1, a=0, b=np.inf),   # Initial tumour volume
    pints.TruncatedGaussianLogPrior(mean=1, sd=1, a=0, b=np.inf),        # Critical volume
    pints.TruncatedGaussianLogPrior(mean=1, sd=1, a=0, b=np.inf),        # Critical volume
    pints.TruncatedGaussianLogPrior(mean=1, sd=1, a=0, b=np.inf),        # Critical volume
    pints.TruncatedGaussianLogPrior(mean=1, sd=1, a=0, b=np.inf),        # Critical volume
    pints.TruncatedGaussianLogPrior(mean=1, sd=1, a=0, b=np.inf),        # Critical volume
    pints.TruncatedGaussianLogPrior(mean=1, sd=1, a=0, b=np.inf),        # Critical volume
    pints.TruncatedGaussianLogPrior(mean=1, sd=1, a=0, b=np.inf),        # Critical volume
    pints.TruncatedGaussianLogPrior(mean=1, sd=1, a=0, b=np.inf),        # Critical volume
    pints.TruncatedGaussianLogPrior(mean=0.1, sd=0.1, a=0, b=np.inf),    # Growth rate
    pints.TruncatedGaussianLogPrior(mean=0.1, sd=0.1, a=0, b=np.inf),    # Growth rate
    pints.TruncatedGaussianLogPrior(mean=0.1, sd=0.1, a=0, b=np.inf),    # Growth rate
    pints.TruncatedGaussianLogPrior(mean=0.1, sd=0.1, a=0, b=np.inf),    # Growth rate
    pints.TruncatedGaussianLogPrior(mean=0.1, sd=0.1, a=0, b=np.inf),    # Growth rate
    pints.TruncatedGaussianLogPrior(mean=0.1, sd=0.1, a=0, b=np.inf),    # Growth rate
    pints.TruncatedGaussianLogPrior(mean=0.1, sd=0.1, a=0, b=np.inf),    # Growth rate
    pints.TruncatedGaussianLogPrior(mean=0.1, sd=0.1, a=0, b=np.inf),    # Growth rate
    pints.TruncatedGaussianLogPrior(mean=0, sd=1, a=0, b=np.inf)]        # Pooled Sigma rel.
# problem.set_data(data)
# problem.set_log_prior(log_priors)
# log_posterior = problem.get_log_posterior()

def fun(log_parameters):
    score, sens = log_posterior.evaluateS1(np.exp(log_parameters))
    return (-score, -sens)

n_runs = 5
ids = log_posterior.get_id()
names = log_posterior.get_parameter_names()
estimates = pd.DataFrame(columns=['ID', 'Parameter', 'Estimate', 'Score', 'Run'])
for run_id in tqdm(range(n_runs)):
    # Run optimisation
    initial_parameters = np.log(erlo.InferenceController(
        log_posterior)._initial_params[0, 0])
    minimizer_kwargs = {"method":"L-BFGS-B", "jac": True}
    result = basinhopping(
        func=fun, x0=initial_parameters, minimizer_kwargs=minimizer_kwargs, niter=10)

    if np.isnan(result.fun) or (result.fun == -np.inf):
        continue

    # Store results
    estimates = estimates.append(pd.DataFrame({
        'ID': ids, 
        'Parameter': names, 
        'Estimate': np.exp(result.x), 
        'Score': result.fun, 
        'Run': run_id}))

/Users/david/Documents/SABS/DPhil/erlotinib/erlotinib/_log_pdfs.py:1227: RuntimeWarning:

An error occured while solving the mechanistic model: 
A numerical error occurred during simulation at t = 0.
Last reached state: 
  myokit.tumour_volume =  4.06992279658902925e-28
Inputs for binding:
  time        = 0.0
  pace        = 0.0
  realtime    = 0.0
  evaluations = 192.0
Function CVode() failed with flag -4 CV_CONV_FAILURE: Convergence test failures occurred too many times during one internal time step or minimum step size was reached..
 A score of -infinity is returned.

/Users/david/Documents/SABS/DPhil/erlotinib/venv/lib/python3.7/site-packages/myokit/_sim/cvodessim.py:654: RuntimeWarning:

CVODES: Internal t = 0.054618 and h = 3.32399e-18 are such that t + h = t on the next step. The solver will continue anyway.

/Users/david/Documents/SABS/DPhil/erlotinib/venv/lib/python3.7/site-packages/myokit/_sim/cvodessim.py:654: RuntimeWarning:

CVODES: Internal t = 0.054618 and h = 2.07749e-1

In [38]:
# Create figure
fig = erlo.plots.ParameterEstimatePlot()
fig.add_data(estimates)
fig.show()